In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Bidirectional,LSTM,Dropout,Dense,Embedding
#IMPORT THE REGULARIZERS
from tensorflow.keras import regularizers
#MULTI TARGET PREDICTION LIBRARIRES CHAINED OUTPUT REGRESSION DUE TO THE DEPENDENCE
from sklearn.multioutput import RegressorChain

df=pd.read_csv("G:/hell.csv")
df2=pd.read_csv("G:/hell2.csv")


df2=df2[df2["lang"]=="en"]
print(df2.head(10))

In [9]:
df.drop(df.columns[0], axis=1,inplace=True)
dt=pd.concat([df["text_short"],df["text_long"],df2["text"]],axis=0)
print(dt)


0         Fortunate to feel calm and relaxed but worried...
1                 Not being able to cuddle my family sucks!
2         Spending time at home with the family and work...
3         I'm OK, I'm feeling positive, but also a littl...
4         Getting very anxious about this whole situatio...
                                ...                        
475250    The only thing surprising about  #covid19 and ...
475253    The Majik of Mind Control and How The Whole Wo...
475254    @HillaryClinton @WHO Hillary!. Perhaps you sho...
475256    Whether it is #Brexit or #Covid19 exceptionali...
475258    It’s been a long day. I’ve been taking my temp...
Length: 269571, dtype: object


In [10]:
tokenizer=Tokenizer(oov_token='OOV')
tokenizer.fit_on_texts(dt)
total_words=len(tokenizer.word_index)+1

input_long=[]
max_len_feed=0
for i in df["text_short"]:
    input_long.append(tokenizer.texts_to_sequences([i])[0])
for i in df["text_long"]:
    if(len(i)>max_len_feed):
        max_len_feed=len(i)
    input_long.append(tokenizer.texts_to_sequences([i])[0])


input_long=np.array(pad_sequences(input_long,maxlen=int(0.45*max_len_feed),padding='pre'))
#PREPARE THE MULTIPLE OUTPUT COLUMNS TO BE PREDICTED
print(len(input_long))
#BELOW CODE FOR CATEGORICAL TRANSFORMATION FOR OUTPUT
import tensorflow.keras.utils as ku
uniques_worry,ids_worry=np.unique(np.array(df["worry"]),return_inverse=True)
output_worry=ku.to_categorical(ids_worry,len(uniques_worry))

uniques_anger,ids_anger=np.unique(np.array(df["anger"]),return_inverse=True)
output_anger=ku.to_categorical(ids_anger,len(uniques_anger))

uniques_fear,ids_fear=np.unique(np.array(df["fear"]),return_inverse=True)
output_fear=ku.to_categorical(ids_fear,len(uniques_fear))

uniques_disgust,ids_disgust=np.unique(np.array(df["disgust"]),return_inverse=True)
output_disgust=ku.to_categorical(ids_disgust,len(uniques_disgust))

uniques_anxiety,ids_anxiety=np.unique(np.array(df["anxiety"]),return_inverse=True)
output_anxiety=ku.to_categorical(ids_anxiety,len(uniques_anxiety))

uniques_sadness,ids_sadness=np.unique(np.array(df["sadness"]),return_inverse=True)
output_sadness=ku.to_categorical(ids_sadness,len(uniques_sadness))

uniques_happiness,ids_happiness=np.unique(np.array(df["happiness"]),return_inverse=True)
output_happiness=ku.to_categorical(ids_happiness,len(uniques_happiness))

uniques_relaxation,ids_relaxation=np.unique(np.array(df["relaxation"]),return_inverse=True)
output_relaxation=ku.to_categorical(ids_relaxation,len(uniques_relaxation))

uniques_desire,ids_desire=np.unique(np.array(df["desire"]),return_inverse=True)
output_desire=ku.to_categorical(ids_desire,len(uniques_desire))
print(str(output_worry)+str(np.array(df["worry"])))
print(str(output_relaxation)+str(np.array(df["relaxation"])))

42519
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]][6 6 6 ... 7 7 7]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]][7 4 6 ... 1 5 2]


In [ ]:
from tensorflow import keras 
#from tensorflow.keras.lay
#CREATE THE BIDIRECTIONAL LSTM MODEL 
#SERIALZIED THE MODEL USING LSTM AND BIDRIECTIONAL LSTM
inputss=keras.Input(shape=(None,),name="multi_feeling",dtype="int32")
#TAKING THE INPUT ABOVE HERE
x=keras.layers.Embedding(total_words, 70, input_length=int(0.45*max_len_feed))(inputss)
x=Bidirectional(LSTM(90, return_sequences = True))(x)
x=Dropout(0.3)(x)
x=LSTM(70)(x)
x=Dense(total_words/4, activation='relu',kernel_regularizer=regularizers.l2(0.01))(x)

worry=Dense(9,activation='softmax')(x)
anger=Dense(9,activation='softmax')(x)
disgust=Dense(9,activation='softmax')(x)
fear=Dense(9,activation='softmax')(x)
anxiety=Dense(9,activation='softmax')(x)
sadness=Dense(9,activation='softmax')(x)
happiness=Dense(9,activation='softmax')(x)
relaxation=Dense(9,activation='softmax')(x)
desire=Dense(9,activation='softmax')(x)
model=keras.Model(inputss,[worry,anger,disgust,fear,anxiety,
                                        sadness,happiness,relaxation,desire])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
print("length max")
print(int(0.65*max_len_feed))

In [ ]:
#CHAIN FED MULTI OUPPUT REGRESSION MODEL
#Wrapper=RegressorChain(model)
history = model.fit(input_long,[output_worry,output_anger,output_disgust,output_fear,output_anxiety,
output_sadness,output_happiness,output_relaxation,output_desire],validation_split=0.28, epochs=25, verbose=1)

In [ ]:
model.save("G:/multi_traget_feeling_precise2_NEWMAINTF.hdf5",include_optimizer=True)

In [ ]:
#PLOT THE ACCURACY AND LOSS GRAPH
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')
plt.figure()
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()